In [17]:
import rasa_nlu
import rasa_core
import spacy

In [28]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- hey there
- let's go
- goodmorning
- goodevening

## intent:goodbye
- see you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- c you later

## intent:mood_affirm
- yes
- Yes
- yep
- indeed
- of course
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- feeling like a king
- wonderful
- I am feeling very good
- I am great
- I am amazing
- so perfect

## intent:mood_curious
- I want to learn
- I am curious
- I want to learn DS
- I want to learn [NLP](group:B)
- I want to learn [CV](group:A)
- [CV](group:A)
- [NLP](group:B)
- Data Science
- curious
- learn
- learn
- wanna learn
- wanna learn DS
- wanna learn [NLP](group:B)
- wanna learn CV
- Good- can I get a link for [NLP](group:B), please?
- Good- can I get a link for [CV](group:A), please?
- Not good. The only thing that could make me fell better is a picture of a cute [kitten](group:cats).

## intent:inform
- [CV](group:A)
- [Computer Vision](group:A)
- [vision](group:A)
- a [image](group:A)
- [NLP](group:B)
- [images](group:A)
- [language](group:B)
- [Natural Language Processing](group:B)
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [29]:
config = """
language: "en"

pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                   # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
""" 

%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


In [30]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
train_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(train_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="curious")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Miniconda3\envs\nikhil\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Miniconda3\envs\nikhil\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Miniconda3\envs\nikhil\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Miniconda3\envs\nikhil\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels w

In [21]:
import json
def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse("I am curious"))

{
  "intent": {
    "name": "mood_curious",
    "confidence": 0.6363038986854366
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "mood_curious",
      "confidence": 0.6363038986854366
    },
    {
      "name": "mood_great",
      "confidence": 0.10867161657399281
    },
    {
      "name": "mood_affirm",
      "confidence": 0.06253292735412333
    },
    {
      "name": "greet",
      "confidence": 0.05678669827389241
    },
    {
      "name": "goodbye",
      "confidence": 0.05287915750598862
    },
    {
      "name": "inform",
      "confidence": 0.04186953729556923
    },
    {
      "name": "mood_deny",
      "confidence": 0.0409561643109971
    }
  ],
  "text": "I am curious"
}


In [20]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)

{'intent_evaluation': {'predictions': [{'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5802712759991918},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5446151768080074},
   {'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5620510986675583},
   {'text': 'hello there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5446151768080074},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.599921957476693},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5636370043031962},
   {'text': 'moin',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5518738574781535},
   {'text': 'hey there',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.5581755541464068},
   {'text': "let's go",
    'intent': 'greet',
    'predicted': 'gree

In [39]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
## curious path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_curious
  - utter_ask_link
* inform{"group":"A"}  
  - action_retrieve_link
  - utter_did_that_help
* mood_affirm
  - utter_happy

## curious path 2
* greet
  - utter_greet
* mood_curious
  - utter_ask_link
* inform{"group":"B"}
  - action_retrieve_link
  - utter_did_that_help
* mood_deny
  - utter_goodbye
  
## curious path 3
* greet
  - utter_greet
* mood_curious{"group":"A"}
  - action_retrieve_link
  - utter_did_that_help
* mood_affirm
  - utter_happy
  
## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## say goodbye
* goodbye
  - utter_goodbye

## fallback
- utter_unclear

"""

%store stories_md > stories.md

Writing 'stories_md' (str) to file 'stories.md'.


In [32]:
domain_yml = """
intents:
- greet
- goodbye
- mood_affirm
- mood_deny
- mood_great
- mood_unhappy
- inform

slots:
  group:
    type: text
    
entities:
- group

actions:
- utter_greet
- utter_did_that_help
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_link
- __main__.ApiAction

templates:
  utter_greet:
  - text: "Hey! How are you?"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  
  utter_happy:
  - text: "Great carry on!"

  utter_goodbye:
  - text: "Bye"
  
  utter_ask_link:
  - text: "To help you learn, I can show you a link of NLP(language processing) or CV(image processing). Which one do you choose?"
"""

%store domain_yml > domain.yml


Writing 'domain_yml' (str) to file 'domain.yml'.


In [36]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display

import requests

class ApiAction(Action):
    def name(self):
        return "action_retrieve_link"

    def run(self, dispatcher, tracker, domain):
        
        group = tracker.get_slot('group')
        if group== 'B':
            response= "http://www.nlp.com/what-is-nlp/"
 
        else:
            response= "https://blog.algorithmia.com/introduction-to-computer-vision/"
            
        dispatcher.utter_message("Here is something to help you up: {}".format(response))

In [33]:

group = tracker.get_slot('group')
r = requests.get('http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true'.format(group))


response = r.content.decode()
response = response.replace('["',"")
response = response.replace('"]',"")
response

NameError: name 'tracker' is not defined

In [40]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

# this will catch predictions the model isn't very certain about
# there is a threshold for the NLU predictions as well as the action predictions
fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)

agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), fallback])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(
    training_data,
    validation_split=0.0,
    epochs=200
)

agent.persist('models/dialogue')

Processed trackers: 100%|█████████████████████████████████████████████| 262/262 [00:14<00:00, 21.03it/s, # actions=274]
Processed actions: 274it [00:00, 491.47it/s, # examples=274]
Processed trackers: 100%|█████████████████████████████████████████████| 262/262 [00:11<00:00, 29.83it/s, # actions=274]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_7 (Masking)          (None, 5, 18)             0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 32)                6528      
_________________________________________________________________
dense_7 (Dense)              (None, 9)                 297       
_________________________________________________________________
activation_7 (Activation)    (None, 9)                 0         
Total params: 6,825
Trainable params: 6,825
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
274/274 [==============================] - ETA: 13s - loss: 2.2533 - acc: 0.09 - ETA: 0s - loss: 2.1930 - acc: 0.1920 - 2s 7ms/step - loss: 2.1860 - acc: 0.2190
Epoch 2/200
274/274 [==============================] - ETA: 0s - loss: 2.1122 - acc: 0.406 - 0s 175

Epoch 53/200
274/274 [==============================] - ETA: 0s - loss: 0.4202 - acc: 0.937 - ETA: 0s - loss: 0.5032 - acc: 0.916 - 0s 328us/step - loss: 0.4806 - acc: 0.9234
Epoch 54/200
274/274 [==============================] - ETA: 0s - loss: 0.3847 - acc: 1.000 - ETA: 0s - loss: 0.4391 - acc: 0.937 - 0s 375us/step - loss: 0.4452 - acc: 0.9234
Epoch 55/200
274/274 [==============================] - ETA: 0s - loss: 0.5379 - acc: 0.875 - ETA: 0s - loss: 0.4837 - acc: 0.895 - 0s 404us/step - loss: 0.4723 - acc: 0.8978
Epoch 56/200
274/274 [==============================] - ETA: 0s - loss: 0.4322 - acc: 0.906 - ETA: 0s - loss: 0.4690 - acc: 0.912 - 0s 397us/step - loss: 0.4554 - acc: 0.9124
Epoch 57/200
274/274 [==============================] - ETA: 0s - loss: 0.5057 - acc: 0.875 - ETA: 0s - loss: 0.4548 - acc: 0.882 - 0s 404us/step - loss: 0.4650 - acc: 0.8796
Epoch 58/200
274/274 [==============================] - ETA: 0s - loss: 0.4573 - acc: 0.937 - ETA: 0s - loss: 0.4729 - acc: 0

Epoch 105/200
274/274 [==============================] - ETA: 0s - loss: 0.1582 - acc: 0.968 - 0s 178us/step - loss: 0.1464 - acc: 0.9745
Epoch 106/200
274/274 [==============================] - ETA: 0s - loss: 0.1727 - acc: 0.968 - 0s 157us/step - loss: 0.1865 - acc: 0.9489
Epoch 107/200
274/274 [==============================] - ETA: 0s - loss: 0.1064 - acc: 1.000 - 0s 124us/step - loss: 0.1375 - acc: 0.9745
Epoch 108/200
274/274 [==============================] - ETA: 0s - loss: 0.0942 - acc: 1.000 - 0s 157us/step - loss: 0.1612 - acc: 0.9635
Epoch 109/200
274/274 [==============================] - ETA: 0s - loss: 0.1434 - acc: 0.937 - 0s 127us/step - loss: 0.1673 - acc: 0.9526
Epoch 110/200
274/274 [==============================] - ETA: 0s - loss: 0.1698 - acc: 1.000 - 0s 175us/step - loss: 0.1672 - acc: 0.9635
Epoch 111/200
274/274 [==============================] - ETA: 0s - loss: 0.1397 - acc: 0.968 - 0s 157us/step - loss: 0.1317 - acc: 0.9745
Epoch 112/200
274/274 [===========

274/274 [==============================] - ETA: 0s - loss: 0.0426 - acc: 1.000 - ETA: 0s - loss: 0.0962 - acc: 0.986 - 0s 302us/step - loss: 0.0988 - acc: 0.9854
Epoch 158/200
274/274 [==============================] - ETA: 0s - loss: 0.1324 - acc: 0.968 - ETA: 0s - loss: 0.0938 - acc: 0.976 - 0s 284us/step - loss: 0.0931 - acc: 0.9781
Epoch 159/200
274/274 [==============================] - ETA: 0s - loss: 0.0645 - acc: 1.000 - ETA: 0s - loss: 0.0674 - acc: 0.988 - 0s 280us/step - loss: 0.0680 - acc: 0.9891
Epoch 160/200
274/274 [==============================] - ETA: 0s - loss: 0.0579 - acc: 1.000 - ETA: 0s - loss: 0.0654 - acc: 0.995 - 0s 328us/step - loss: 0.0603 - acc: 0.9964
Epoch 161/200
274/274 [==============================] - ETA: 0s - loss: 0.0362 - acc: 1.000 - ETA: 0s - loss: 0.0787 - acc: 0.974 - 0s 371us/step - loss: 0.0697 - acc: 0.9818
Epoch 162/200
274/274 [==============================] - ETA: 0s - loss: 0.0670 - acc: 1.000 - ETA: 0s - loss: 0.0673 - acc: 0.991 - 0

In [42]:
from rasa_core.agent import Agent

agent = Agent.load('models/dialogue', interpreter=model_directory)
print("Your bot is ready to talk! Type your messages here or send 'stop'")

while True:
    a = input()
    if a.lower() == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
Hey
Hey! How are you?
Wanna learn
To help you learn, I can show you a link of NLP(language processing) or CV(image processing). Which one do you choose?
NLP
Here is something to help you up: http://www.nlp.com/what-is-nlp/
Did that help you?
no
Bye
Bye
Bye
stop


In [37]:
from rasa_core.agent import Agent

# agent = Agent.load('models/dialogue', interpreter=model_directory)
# print("Your bot is ready to talk! Type your messages here or send 'stop'")

a = "Hello, I'm great"

responses = agent.handle_message(a)

for response in responses:
    print("Chatbot says: ",response["text"])


# print(agent.handle_message('I am sad')[0]["text"])

Chatbot says:  To cheer you up, I can show you a cute picture of a dog, cat or a bird. Which one do you choose?
